In [1]:
import pandas as pd
from decouple import config
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta, time
import time 
from sqlalchemy import text
import oracledb
import sys

In [2]:
query = """SELECT
--ras.REDASISCOD,
--ras.REDASISDES,
cas.CENASICOD,
--cas.CENASIDES,
aho.AREHOSCOD,
--aho.AREHOSDES,
sho.SERVHOSCOD,
--sho.SERVHOSDES,
--act.ACTCOD,
--act.ACTDES,
--ace.ACTESPCOD,
--ace.ACTESPNOM,
act.ACTCOD || ace.ACTESPCOD AS ACTESPCOD,
med.PERNACFEC AS MEDNACFEC,
med.PERSEXOCOD AS MEDSEXO,
goc.GRUPOCUPCOD,
--goc.GRUPOCUPNOM,
cot.CONDTRABCOD,
--cot.CONDTRABNOM,
pac.PERDOCIDENNUM,
pac.PERNACFEC AS PACNACFEC,
pac.PERSEXOCOD AS PACSEXO,
cit.CITAMBPROCONFEC,
cit.CITAMBCREFEC,
cit.CITAMBHORCIT,
tci.TIPOCITACOD,
--tci.TIPOCITANOM,
coc.CONDCITACOD,
--coc.CONDCITANOM,
moc.MODOTORCITACOD,
--moc.MODOTORCITANOM,
eci.ESTCITCOD
--eci.ESTCITNOM
FROM
CTCAM10 cit
LEFT OUTER JOIN
	CMCAS10 cas
	ON cit.CITAMBCENASICOD = cas.CENASICOD
LEFT OUTER JOIN
	CMRAS10 ras
	ON cas.REDASISCOD = ras.REDASISCOD
LEFT OUTER JOIN
	CMAHO10 aho
	ON cit.CITAMBAREHOSCOD = aho.AREHOSCOD
LEFT OUTER JOIN
	CMSHO10 sho
	ON cit.CITAMBSERVHOSCOD = sho.SERVHOSCOD
LEFT OUTER JOIN
	CMACT10 act
	ON cit.CITAMBACTCOD = act.ACTCOD
LEFT OUTER JOIN
	CMACE10 ace
	ON cit.CITAMBACTCOD = ace.ACTCOD
	AND cit.CITAMBACTESPCOD = ace.ACTESPCOD
LEFT OUTER JOIN
	CMPER10 med
	ON cit.CITAMBPERASISDOCIDENNUM = med.PERDOCIDENNUM
LEFT OUTER JOIN
	CMPRS10 prs
	ON cit.CITAMBPERASISDOCIDENNUM = prs.PERASISDOCIDENNUM
LEFT OUTER JOIN
	CBGOC10 goc
	ON prs.GRUPOCUPCOD = goc.GRUPOCUPCOD
LEFT OUTER JOIN
	CBCOT10 cot
	ON prs.CONDTRABCOD = cot.CONDTRABCOD
LEFT OUTER JOIN
	CBTCI10 tci
	ON cit.TIPOCITACOD = tci.TIPOCITACOD
LEFT OUTER JOIN
	CBCOC10 coc
	ON cit.CONDCITACOD = coc.CONDCITACOD
LEFT OUTER JOIN
	CBMOC10 moc
	ON cit.MODOTORCITACOD = moc.MODOTORCITACOD
LEFT OUTER JOIN
	CBECI10 eci
	ON cit.ESTCITCOD = eci.ESTCITCOD
LEFT OUTER JOIN
	CMPER10 pac
	ON cit.CITAMBPACSECNUM = pac.PERSECNUM
WHERE
	act.ACTCOD = '91'
	AND cit.CITAMBPROCONFEC >= TO_DATE('2024-07-01', 'YYYY-MM-DD')
	AND cit.CITAMBPROCONFEC < TO_DATE('2024-07-03', 'YYYY-MM-DD')
	AND eci.ESTCITCOD IN ('4', '5')
    AND ras.REDASISCOD IN ('07')
"""

In [3]:
oracledb.init_oracle_client()
oracledb.version = "8.3.0"
sys.modules["cx_Oracle"] = oracledb
un = config("USER4_BDI_POSTGRES")
pw = config("PASS4_BDI_POSTGRES")
hostname=config("HOST4_BDI_POSTGRES")
service_name="WNET"
port = 1527

engine0 = create_engine(f'oracle://{un}:{pw}@',connect_args={
        "host": hostname,
        "port": port,
        "service_name": service_name
    }
)

connection0 = engine0.connect()

data2 = pd.read_sql_query(query, con=connection0)

In [4]:
data2.columns = data2.columns.str.upper()

In [5]:
#DUMMIES
#query_REDASISCOD = 'SELECT REDASISCOD FROM CMRAS10'
#REDASISCOD = pd.read_sql_query(query_REDASISCOD, con=connection0)
query_CENASICOD = 'SELECT CENASICOD FROM CMCAS10'
CENASICOD = pd.read_sql_query(query_CENASICOD, con=connection0)
query_AREHOSCOD = 'SELECT AREHOSCOD FROM CMAHO10'
AREHOSCOD = pd.read_sql_query(query_AREHOSCOD, con=connection0)
query_SERVHOSCOD = 'SELECT SERVHOSCOD FROM CMSHO10'
SERVHOSCOD = pd.read_sql_query(query_SERVHOSCOD, con=connection0)
query_ACTESPCOD = "SELECT ACTCOD || ACTESPCOD AS ACTESPCOD FROM CMACE10 WHERE ACTCOD = '91'"
ACTESPCOD = pd.read_sql_query(query_ACTESPCOD, con=connection0)
query_GRUPOCUPCOD = 'SELECT GRUPOCUPCOD FROM CBGOC10'
GRUPOCUPCOD = pd.read_sql_query(query_GRUPOCUPCOD, con=connection0)
query_CONDTRABCOD = 'SELECT CONDTRABCOD FROM CBCOT10'
CONDTRABCOD = pd.read_sql_query(query_CONDTRABCOD, con=connection0)
query_TIPOCITACOD = 'SELECT TIPOCITACOD FROM CBTCI10'
TIPOCITACOD = pd.read_sql_query(query_TIPOCITACOD, con=connection0)
query_CONDCITACOD = 'SELECT CONDCITACOD FROM CBCOC10'
CONDCITACOD = pd.read_sql_query(query_CONDCITACOD, con=connection0)
query_MODOTORCITACOD = 'SELECT MODOTORCITACOD FROM CBMOC10'
MODOTORCITACOD = pd.read_sql_query(query_MODOTORCITACOD, con=connection0)

connection0.close()

In [6]:
#REDASISCOD.columns = REDASISCOD.columns.str.upper()
CENASICOD.columns = CENASICOD.columns.str.upper()
AREHOSCOD.columns = AREHOSCOD.columns.str.upper()
SERVHOSCOD.columns = SERVHOSCOD.columns.str.upper()
ACTESPCOD.columns = ACTESPCOD.columns.str.upper()
GRUPOCUPCOD.columns = GRUPOCUPCOD.columns.str.upper()
CONDTRABCOD.columns = CONDTRABCOD.columns.str.upper()
TIPOCITACOD.columns = TIPOCITACOD.columns.str.upper()
CONDCITACOD.columns = CONDCITACOD.columns.str.upper()
MODOTORCITACOD.columns = MODOTORCITACOD.columns.str.upper()

In [7]:
#REDASISCOD = pd.get_dummies(REDASISCOD)
CENASICOD = pd.get_dummies(CENASICOD)
AREHOSCOD = pd.get_dummies(AREHOSCOD)
SERVHOSCOD = pd.get_dummies(SERVHOSCOD)
ACTESPCOD = pd.get_dummies(ACTESPCOD)
GRUPOCUPCOD = pd.get_dummies(GRUPOCUPCOD)
CONDTRABCOD = pd.get_dummies(CONDTRABCOD)
TIPOCITACOD = pd.get_dummies(TIPOCITACOD)
CONDCITACOD = pd.get_dummies(CONDCITACOD)
MODOTORCITACOD = pd.get_dummies(MODOTORCITACOD)

In [8]:
# Extraer las columnas de cada dataframe dummy
#REDASISCOD_columns = REDASISCOD.columns.tolist()
CENASICOD_columns = CENASICOD.columns.tolist()
AREHOSCOD_columns = AREHOSCOD.columns.tolist()
SERVHOSCOD_columns = SERVHOSCOD.columns.tolist()
ACTESPCOD_columns = ACTESPCOD.columns.tolist()
GRUPOCUPCOD_columns = GRUPOCUPCOD.columns.tolist()
CONDTRABCOD_columns = CONDTRABCOD.columns.tolist()
TIPOCITACOD_columns = TIPOCITACOD.columns.tolist()
CONDCITACOD_columns = CONDCITACOD.columns.tolist()
MODOTORCITACOD_columns = MODOTORCITACOD.columns.tolist()

# Crear una lista con los nombres de las columnas básicas
basic_columns = ['MEDSEXO', 'PACSEXO', 'HORA_CITA', 'EDAD_MEDICO', 'EDAD_PACIENTE', 
                'DIAS_ENTRE_CITA', 'NUM_CITAS', 'NUM_DESERCIONES', 'TASA_DESERCION']

# Combinar todas las columnas en una sola lista
# all_columns = (basic_columns + REDASISCOD_columns + CENASICOD_columns + AREHOSCOD_columns + 
#             SERVHOSCOD_columns + ACTESPCOD_columns + GRUPOCUPCOD_columns + 
#             CONDTRABCOD_columns + TIPOCITACOD_columns + CONDCITACOD_columns + MODOTORCITACOD_columns)

all_columns = (basic_columns + CENASICOD_columns + AREHOSCOD_columns + 
            SERVHOSCOD_columns + ACTESPCOD_columns + GRUPOCUPCOD_columns + 
            CONDTRABCOD_columns + TIPOCITACOD_columns + CONDCITACOD_columns + MODOTORCITACOD_columns)


# Crear un DataFrame vacío con todas las columnas
df_model = pd.DataFrame(columns=all_columns)

In [9]:
#data2 = data2.drop(['REDASISCOD', 'CENASICOD', 'AREHOSCOD' , 'SERVHOSCOD','ACTCOD', 'ACTESPCOD','GRUPOCUPCOD', 'CONDTRABCOD','TIPOCITACOD', 'CONDCITACOD', 'MODOTORCITACOD', 'ESTCITCOD'],axis=1)
#data2 = data2.drop(['REDASISDES', 'CENASIDES', 'AREHOSDES', 'SERVHOSDES', 'ACTDES',  'ACTESPNOM', 'GRUPOCUPNOM', 'CONDTRABNOM', 'TIPOCITANOM', 'CONDCITANOM', 'MODOTORCITANOM', 'ESTCITNOM'],axis=1)

In [10]:
# Filtrar fechas que están fuera del rango razonable (por ejemplo, antes de 1900 y después de 2100)
def filter_dates(date_series, min_year=1900, max_year=2100):
    date_series = pd.to_datetime(date_series, errors='coerce')
    return date_series[(date_series.dt.year >= min_year) & (date_series.dt.year <= max_year)]

data2['MEDNACFEC'] = filter_dates(data2['MEDNACFEC'])
data2['PACNACFEC'] = filter_dates(data2['PACNACFEC'])
data2['CITAMBCREFEC'] = filter_dates(data2['CITAMBCREFEC'])
data2['CITAMBPROCONFEC'] = filter_dates(data2['CITAMBPROCONFEC'])

In [11]:
print(data2.shape)
# Convertir fechas a tipo datetime, ignorando errores y manteniendo valores no convertibles como NaT
data2['MEDNACFEC'] = pd.to_datetime(data2['MEDNACFEC'], errors='coerce')
data2['PACNACFEC'] = pd.to_datetime(data2['PACNACFEC'], errors='coerce')
data2['CITAMBCREFEC'] = pd.to_datetime(data2['CITAMBCREFEC'], errors='coerce')
data2['CITAMBPROCONFEC'] = pd.to_datetime(data2['CITAMBPROCONFEC'], errors='coerce')
print(data2.shape)


(24453, 18)
(24453, 18)


In [12]:
# Extraer solo la hora de la columna CITAMBHORCIT
def extract_time_from_string(time_str):
    try:
        # Si es un objeto datetime, extraer la hora directamente
        if isinstance(time_str, datetime):
            return time_str.time()
        # Si es una cadena de texto, procesarla
        if ' ' in time_str:
            time_str = time_str.split(' ')[1]
        return datetime.strptime(time_str, '%H:%M:%S').time()
    except (ValueError, TypeError):
        return None

data2['CITAMBHORCIT'] = data2['CITAMBHORCIT'].apply(extract_time_from_string)

In [13]:
data2['HORA_CITA'] = data2['CITAMBHORCIT'].apply(lambda x: x.hour * 60 + x.minute if pd.notnull(x) else None)

In [14]:
# Calcular la edad del médico y del paciente
data2['EDAD_MEDICO'] = (pd.to_datetime('today') - data2['MEDNACFEC']).dt.days // 365
data2['EDAD_PACIENTE'] = (pd.to_datetime('today') - data2['PACNACFEC']).dt.days // 365

# Calcular el tiempo entre la fecha de otorgamiento y la fecha de la cita
data2['DIAS_ENTRE_CITA'] = (data2['CITAMBPROCONFEC'] - data2['CITAMBCREFEC']).dt.days + 1

# Calcular el precedente del paciente
data2['NUM_CITAS'] = data2.groupby('PERDOCIDENNUM')['PERDOCIDENNUM'].transform('count')
data2['NUM_DESERCIONES'] = data2.groupby('PERDOCIDENNUM')['ESTCITCOD'].transform(lambda x: (x == '5').sum())
data2['TASA_DESERCION'] = data2['NUM_DESERCIONES'] / data2['NUM_CITAS']

# Eliminar columnas originales que no serán necesarias
data2 = data2.drop(columns=['MEDNACFEC', 'PACNACFEC', 'CITAMBCREFEC', 'CITAMBPROCONFEC', 'CITAMBHORCIT', 'PERDOCIDENNUM'])

# Verificar y manejar valores faltantes después de todas las transformaciones
data2 = data2.dropna()

In [15]:
# Codificación One-Hot de variables categóricas
#data2 = pd.get_dummies(data2, columns=['REDASISDES', 'CENASIDES', 'AREHOSDES', 'SERVHOSDES', 'ACTDES',  'ACTESPDES', 'GRUPOCUPNOM', 'CONDTRABNOM', 'TIPOCITANOM', 'CONDCITANOM', 'MODOTORCITANOM'])
#data2 = pd.get_dummies(data2, columns=['REDASISCOD', 'CENASICOD', 'AREHOSCOD' , 'SERVHOSCOD', 'ACTESPCOD','GRUPOCUPCOD', 'CONDTRABCOD','TIPOCITACOD', 'CONDCITACOD', 'MODOTORCITACOD'])
data2 = pd.get_dummies(data2, columns=['CENASICOD', 'AREHOSCOD' , 'SERVHOSCOD', 'ACTESPCOD','GRUPOCUPCOD', 'CONDTRABCOD','TIPOCITACOD', 'CONDCITACOD', 'MODOTORCITACOD'])

# Limpiar nombres de columnas
data2.columns = data2.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Separar características (X) y variable objetivo (y)
X = data2.drop(columns=['ESTCITCOD'])
y = data2['ESTCITCOD'].apply(lambda x: 1 if x == '5' else 0)

In [16]:
import gc
# Eliminar el DataFrame
del data2
#del REDASISCOD
del CENASICOD
del AREHOSCOD
del SERVHOSCOD
del ACTESPCOD
del GRUPOCUPCOD
del CONDTRABCOD
del TIPOCITACOD
del CONDCITACOD
del MODOTORCITACOD
gc.collect()

7

In [17]:
# Definir el tamaño del bloque
block_size = 50  # Ajusta según la memoria disponible

# Asegurar que X tenga todas las columnas de df_model en bloques
missing_cols = [col for col in df_model.columns if col not in X.columns]

# Procesar y agregar bloques de columnas faltantes
for i in range(0, len(missing_cols), block_size):
    block = missing_cols[i:i + block_size]
    for col in block:
        X[col] = 0

# Asegurar el orden de las columnas en bloques
# No es necesario dividir en bloques aquí, ya que solo estamos reordenando las columnas
X = X[df_model.columns]

/tmp/ipykernel_2529393/575242985.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = 0
/tmp/ipykernel_2529393/575242985.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = 0
/tmp/ipykernel_2529393/575242985.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = 0
/tmp/ipyke

In [18]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
# Eliminar el DataFrame
del X
del y

gc.collect()

0

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de regresión logística
log_reg = LogisticRegression(max_iter=1100)
log_reg.fit(X_train, y_train)

# Evaluar el modelo
y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

print("Regresión Logística")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))                              


/home/ugadingenieria01/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Regresión Logística
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.96      0.96       461

    accuracy                           1.00      7331
   macro avg       0.98      0.98      0.98      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9998503348520873


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de regresión logística
log_reg = LogisticRegression(solver='saga', max_iter=500)
log_reg.fit(X_train, y_train)

# Evaluar el modelo
y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

print("Regresión Logística")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))                              


/home/ugadingenieria01/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Regresión Logística
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6870
           1       1.00      0.00      0.01       461

    accuracy                           0.94      7331
   macro avg       0.97      0.50      0.49      7331
weighted avg       0.94      0.94      0.91      7331

ROC AUC Score: 0.9407016579993497


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de regresión logística
log_reg = LogisticRegression(solver='lbfgs', max_iter=500)
log_reg.fit(X_train, y_train)

# Evaluar el modelo
y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

print("Regresión Logística")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))                              


/home/ugadingenieria01/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Regresión Logística
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.96      0.96       461

    accuracy                           1.00      7331
   macro avg       0.98      0.98      0.98      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9998515978491161


In [23]:
from sklearn.tree import DecisionTreeClassifier

# Entrenar el modelo de árbol de decisión
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# Evaluar el modelo
y_pred = tree.predict(X_test)
y_prob = tree.predict_proba(X_test)[:, 1]

print("Árbol de Decisión")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))


Árbol de Decisión
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.98      0.96      0.97       461

    accuracy                           1.00      7331
   macro avg       0.99      0.98      0.98      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9796766411857016


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de random forest
forest = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    max_depth=30,
    class_weight={0: 1, 1: 5}
)
forest.fit(X_train, y_train)

# Evaluar el modelo
y_pred = forest.predict(X_test)
y_prob = forest.predict_proba(X_test)[:, 1]

print("Random Forest")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.98      0.98       461

    accuracy                           1.00      7331
   macro avg       0.98      0.99      0.99      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9997884479976761


In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de random forest
forest = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    max_depth=30,
    class_weight={0: 1, 1: 3}
)
forest.fit(X_train, y_train)

# Evaluar el modelo
y_pred = forest.predict(X_test)
y_prob = forest.predict_proba(X_test)[:, 1]

print("Random Forest")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.98      0.97       461

    accuracy                           1.00      7331
   macro avg       0.98      0.99      0.98      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9997991834724208


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Entrenar el modelo de random forest
forest = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    max_depth=30
)
forest.fit(X_train, y_train)

# Evaluar el modelo
y_pred = forest.predict(X_test)
y_prob = forest.predict_proba(X_test)[:, 1]

print("Random Forest")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.97      0.97       461

    accuracy                           1.00      7331
   macro avg       0.98      0.98      0.98      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9998392836280853


In [27]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Balancear los datos con SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Entrenar el modelo con los datos balanceados
forest.fit(X_resampled, y_resampled)

# Evaluar el modelo
y_pred = forest.predict(X_test)
y_prob = forest.predict_proba(X_test)[:, 1]

print("Random Forest with SMOTE")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))


Random Forest with SMOTE
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.97      0.98      0.97       461

    accuracy                           1.00      7331
   macro avg       0.98      0.99      0.99      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9997177201640632


In [28]:
from sklearn.ensemble import VotingClassifier

# Definir los modelos base
clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf2 = RandomForestClassifier(n_estimators=200, random_state=42)
clf3 = RandomForestClassifier(n_estimators=300, random_state=42)

# Crear el ensamblado de votación
ensemble = VotingClassifier(estimators=[('rf1', clf1), ('rf2', clf2), ('rf3', clf3)], voting='soft')

# Entrenar el ensamblado
ensemble.fit(X_resampled, y_resampled)

# Evaluar el ensamblado
y_pred_ensemble = ensemble.predict(X_test)
y_prob_ensemble = ensemble.predict_proba(X_test)[:, 1]

print("Ensemble Classifier")
print(classification_report(y_test, y_pred_ensemble))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_ensemble))


Ensemble Classifier
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.96      0.98      0.97       461

    accuracy                           1.00      7331
   macro avg       0.98      0.99      0.99      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9997357178717238


In [29]:
from sklearn.model_selection import GridSearchCV

# Definir los parámetros para el grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'class_weight': [{0: 1, 1: 1},{0: 1, 1: 5}, {0: 1, 1: 10}]
}

# Crear el grid search
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_forest = grid_search.best_estimator_

# Evaluar el mejor modelo
y_pred_best = best_forest.predict(X_test)
y_prob_best = best_forest.predict_proba(X_test)[:, 1]

print("Best Random Forest")
print(classification_report(y_test, y_pred_best))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob_best))

Best Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6870
           1       0.96      0.99      0.97       461

    accuracy                           1.00      7331
   macro avg       0.98      0.99      0.99      7331
weighted avg       1.00      1.00      1.00      7331

ROC AUC Score: 0.9996432033393642


In [30]:
print(grid_search.best_params_)

{'class_weight': {0: 1, 1: 5}, 'max_depth': 20, 'n_estimators': 200}


In [31]:
# import xgboost as xgb

# # Entrenar el modelo de XGBoost
# xgboost = xgb.XGBClassifier(use_label_encoder=False, enable_categorical=True, eval_metric='logloss')
# xgboost.fit(X_train, y_train)

# # Evaluar el modelo
# y_pred = xgboost.predict(X_test)
# y_prob = xgboost.predict_proba(X_test)[:, 1]

# print("XGBoost")
# print(classification_report(y_test, y_pred))
# print("ROC AUC Score:", roc_auc_score(y_test, y_prob))


In [32]:
# import lightgbm as lgb

# # Entrenar el modelo de LightGBM
# lgbm = lgb.LGBMClassifier()
# lgbm.fit(X_train, y_train)

# # Evaluar el modelo
# y_pred = lgbm.predict(X_test)
# y_prob = lgbm.predict_proba(X_test)[:, 1]

# print("LightGBM")
# print(classification_report(y_test, y_pred))
# print("ROC AUC Score:", roc_auc_score(y_test, y_prob))


In [33]:
# from sklearn.svm import SVC

# # Entrenar el modelo de SVM
# svm = SVC(probability=True)
# svm.fit(X_train, y_train)

# # Evaluar el modelo
# y_pred = svm.predict(X_test)
# y_prob = svm.predict_proba(X_test)[:, 1]

# print("SVM")
# print(classification_report(y_test, y_pred))
# print("ROC AUC Score:", roc_auc_score(y_test, y_prob))


In [34]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros para Random Forest:", grid_search.best_params_)
print("Mejor ROC AUC Score para Random Forest:",     grid_search.best_score_)


Mejores hiperparámetros para Random Forest: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 200}
Mejor ROC AUC Score para Random Forest: 0.999883107099927


In [35]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}

random_search = RandomizedSearchCV(estimator=xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'), param_distributions=param_distributions, n_iter=100, cv=5, scoring='roc_auc', random_state=42)
random_search.fit(X_train, y_train)

print("Mejores hiperparámetros para XGBoost:", random_search.best_params_)
print("Mejor ROC AUC Score para XGBoost:", random_search.best_score_)


/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


ValueError: 
All the 500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ugadingenieria01/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/sklearn.py", line 1471, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/home/ugadingenieria01/.local/lib/python3.10/site-packages/xgboost/sklearn.py", line 448, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  Fi

In [ ]:
import joblib
# Exportar el modelo a un archivo .joblib
joblib.dump(forest, 'modelo_random_forest_SDC.joblib')

['modelo_random_forest_SDC.joblib']